In [16]:
import os
from dotenv import load_dotenv
from igdb.wrapper import IGDBWrapper
import json
import pandas as pd

load_dotenv()
ID = os.getenv('CLIENTID')
TOKEN = os.getenv('ACCESSTOKEN')

wrapper = IGDBWrapper(ID, TOKEN)

['scooby-snacks', 'destructible cover']

In [47]:
import time

keys = ['id', 'category', 'rating', 'rating_count', 'aggregated_rating', \
        'first_release_date', 'follows', 'game_modes', \
        'genres', 'involved_companies', 'keywords', 'summary', 'storyline', 'multiplayer_modes', \
        'name', 'platforms', 'player_perspectives', 'similar_games', 'themes']

batch = {k: [] for k in keys}

byte_array = wrapper.api_request(
            'games',
            'fields id, name, rating, aggregated_rating, category, storyline, \
             first_release_date, follows, rating_count, game_modes, genres, involved_companies,\
             keywords, multiplayer_modes, platforms, summary, player_perspectives,\
             similar_games, themes, expanded_games; limit 500; \
             where category = 0 & first_release_date > 1; sort first_release_date;'
          )

count = 1

modfields = ['game_modes', 'genres', 'platforms', 'involved_companies', 'keywords',\
              'multiplayer_modes', 'player_perspectives', 'similar_games', 'themes']

while len(byte_array) > 2:
        
    for i in json.loads(byte_array.decode('utf-8')):
                              
        for modfield in modfields:
            try:
                i[modfield] = ' '.join([str(s) for s in i[modfield]])
            except KeyError:
                i[modfield] = ''
    
        for key in keys:
            try:
                batch[key].append(i[key])
            except KeyError:
                batch[key].append(None)
            
    time.sleep(0.25)
    
    byte_array = wrapper.api_request(
            'games',
            f'fields id, name, rating, aggregated_rating, category, storyline,\
             first_release_date, follows, rating_count, summary, game_modes, genres, involved_companies,\
             keywords, multiplayer_modes, platforms, player_perspectives,\
             similar_games, themes, expanded_games; limit 500; offset {500*count}; \
             where category = 0 & first_release_date > 1; sort first_release_date;'
          )
    count += 1


base = pd.DataFrame(batch)

In [54]:
base.set_index(['id'], inplace=True)

In [27]:
byte_array = wrapper.api_request(
            'keywords',
            f'fields name; limit 500;'
          )

count = 1

keyworddict = {'id': [], 'name':[]}

while len(byte_array) > 2:
        
    for i in json.loads(byte_array.decode('utf-8')):
        keyworddict['id'].append(i['id'])
        keyworddict['name'].append(i['name'])
    
    time.sleep(0.25)
    
    byte_array = wrapper.api_request(
            'keywords',
            f'fields name; limit 500; offset {500*count};'
          )
    
    count += 1
        
keywords = pd.DataFrame(keyworddict)


In [56]:
keywords.set_index(['id'],inplace=True)

In [37]:
byte_array = wrapper.api_request(
            'multiplayer_modes',
            'fields *;exclude checksum, game; limit 500;'
          )

count = 1

keys =  ['id', 'campaigncoop', 'dropin', 'lancoop', 'offlinecoop', \
         'offlinecoopmax', 'offlinemax', 'onlinecoop', 'onlinecoopmax', \
         'onlinemax', 'platform', 'splitscreen']

multiplayermodesdict = {k: [] for k in keys}

while len(byte_array) > 2:
    
    for i in json.loads(byte_array.decode('utf-8')):
        for key in keys:
            try:
                multiplayermodesdict[key].append(i[key])
            except KeyError:
                multiplayermodesdict[key].append(None)
    
    time.sleep(0.25)
    
    byte_array = wrapper.api_request(
            'multiplayer_modes',
            f'fields *;exclude checksum, game; limit 500; offset {500*count};'
          )
    
    count += 1
        
multimodes = pd.DataFrame(multiplayermodesdict)

In [55]:
multimodes.set_index(['id'], inplace=True)

In [64]:
pd.Series().values

[558, 732, 947, 970, 1025, 1299, 1440, 4138, 4754, 4992, 5442, 8241]

In [79]:
def keywordsidtoname(ids: str):
    if len(ids) > 0:
        try:
            names = ' '.join(keywords.loc[[int(i) for i in ids.split()], 'name'].values)
        except KeyError:
            names = ''
    else:
        names = ''
    return names

base['keywords'] = base['keywords'].apply(keywordsidtoname)

In [80]:
base

,category,rating,rating_count,aggregated_rating,first_release_date,follows,game_modes,genres,involved_companies,keywords,summary,storyline,multiplayer_modes,name,platforms,player_perspectives,similar_games,themes
id,,,,,,,,,,,,,,,,,,
56115,0,NaN,NaN,NaN,15552000,NaN,,5,,modern warfare exploration jet first person sh...,"An early 3D flight simulator game, released by...",None,,Jet Rocket,52,1,8559 10964 11171 15536 28751 57217 58075 64311...,1
11304,0,NaN,NaN,NaN,21945600,NaN,1,,91161 91162,western text-based text adventure,Highnoon is a BASIC game developed by Christop...,None,,Highnoon,82 107,5,18290 58867 62473 79064 100800 103301 103302 1...,1
76506,0,NaN,NaN,NaN,31449600,NaN,1,13 32,150176 150177,,"The Game of Life, also known simply as Life, i...",None,,Game of Life,6 69 91 103,,28277 33603 37419 44087 51577 65827 76263 7913...,
11485,0,NaN,NaN,50.0,47174400,NaN,1,5 15 30,50077,pinball spaceship sci-fi shooter discrete logic,Star Trek is a text-based computer game that p...,None,,Star Trek,104 106,5,13200 13660 17130 24620 34823 55038 57187 6828...,18
11396,0,NaN,NaN,NaN,52531200,3.0,2,5,46322,spaceship sci-fi shooter record breakers,Galaxy Game is one of the earliest known coin-...,None,,Galaxy Game,108,3,17130 24620 55038 57187 68286 68287 78770 1032...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148057,0,NaN,NaN,NaN,1746057600,NaN,1,2 9 31 32 34,135411,indie retro point and click point and click ad...,Pandorex is a Point & Click Adventure featurin...,Pandorex.\nNobody really knows what is up with...,,Pandorex,6,4,236 11367 18011 18841 24426 25222 26870 27725 ...,17 27 43
165549,0,NaN,NaN,NaN,1767139200,NaN,,34,146855,,The sequel would be a direct continuation of t...,None,,Butterfly Soup 2,6 14,5,19075 135063 137068 140022 141888 144502 14510...,27 44
214912,0,NaN,NaN,NaN,1767139200,NaN,1,12,185357,,Penny Blood is a alternative history horror JR...,None,,Penny Blood,6 167 169,2,27092 54775 69007 96217 99118 106987 113636 11...,17 19 22 43


In [23]:
import os.path

datasetdir = os.path.join('.', 'dataset')
filename = 'games.csv'
path = os.path.join(datasetdir, filename)
base.to_csv(path, index=False)

In [34]:
byte_array = wrapper.api_request(
            'multiplayer_modes',
            'fields *;exclude checksum, game; limit 500;sort id;')
for i,j in enumerate(json.loads(byte_array.decode('utf-8'))):
    print(i, j)



0 {'id': 5, 'campaigncoop': False, 'dropin': False, 'lancoop': False, 'offlinecoop': False, 'onlinecoop': False, 'onlinemax': 2, 'platform': 6, 'splitscreen': False}
1 {'id': 6, 'campaigncoop': False, 'dropin': False, 'lancoop': False, 'offlinecoop': False, 'onlinecoop': False, 'onlinemax': 2, 'platform': 14, 'splitscreen': False}
2 {'id': 7, 'campaigncoop': False, 'dropin': False, 'lancoop': False, 'offlinecoop': False, 'offlinemax': 30, 'onlinecoop': False, 'platform': 6, 'splitscreen': False}
3 {'id': 8, 'campaigncoop': False, 'dropin': False, 'lancoop': False, 'offlinecoop': False, 'onlinecoop': False, 'platform': 6, 'splitscreen': False}
4 {'id': 12, 'campaigncoop': False, 'dropin': False, 'lancoop': False, 'offlinecoop': False, 'offlinecoopmax': 6, 'onlinecoop': False, 'splitscreen': False}
5 {'id': 13, 'campaigncoop': False, 'dropin': False, 'lancoop': False, 'offlinecoop': False, 'offlinemax': 2, 'onlinecoop': False, 'onlinemax': 2, 'platform': 48, 'splitscreen': False}
6 {'id'